In [37]:
import sys 
import os  
import multiprocessing
import datetime as dt 
import pandas as pd 
import numpy as np 
import talib
import sqlite3 as sql
import matplotlib.pyplot as plt
from itertools import repeat
from p_tqdm import p_umap
sys.path.append('../')
from backtester import * 
from fxcmtoken import my_assets

In [2]:
for i, a in enumerate(my_assets):
    print(i, a)

0 EUR/USD
1 USD/JPY
2 GBP/USD
3 USD/CHF
4 AUD/USD
5 USD/CAD
6 NZD/USD
7 AUS200
8 ESP35
9 EUSTX50
10 FRA40
11 GER30
12 HKG33
13 JPN225
14 NAS100
15 SPX500
16 UK100
17 US30
18 VOLX
19 Copper
20 NGAS
21 UKOil
22 USOil
23 XAU/USD
24 XAG/USD


In [3]:
freq = 'm5'
conn = sql.connect(f'../PriceData/PriceData_{freq}.db')
ticker = my_assets[0]
print(ticker)
data_sql = pd.read_sql(f"SELECT * FROM '{ticker}'", conn, parse_dates=['date'], index_col=['date'])
for col in ['open', 'close', 'high', 'low']:
    data_sql[col] = data_sql[['bid'+col, 'ask'+col]].mean(axis=1)
conn.close()
data_sql.tail()

EUR/USD


,bidopen,bidclose,bidhigh,bidlow,askopen,askclose,askhigh,asklow,tickqty,open,close,high,low
date,,,,,,,,,,,,,
2022-07-22 20:35:00,1.02095,1.02124,1.02124,1.02095,1.02111,1.02140,1.02140,1.02111,128,1.02103,1.021320,1.021320,1.021030
2022-07-22 20:40:00,1.02123,1.02135,1.02155,1.02112,1.02141,1.02148,1.02168,1.02130,303,1.02132,1.021415,1.021615,1.021210
2022-07-22 20:45:00,1.02135,1.02111,1.02135,1.02085,1.02149,1.02130,1.02150,1.02104,267,1.02142,1.021205,1.021425,1.020945
2022-07-22 20:50:00,1.02112,1.02100,1.02115,1.02087,1.02132,1.02125,1.02137,1.02104,397,1.02122,1.021125,1.021260,1.020955
2022-07-22 20:55:00,1.02100,1.02137,1.02144,1.02094,1.02126,1.02169,1.02173,1.02124,140,1.02113,1.021530,1.021585,1.021090


In [6]:
def get_performance(data, split_date, params, freq):
    # params --> (mama, slope, tsf)
      
    data['ht'] = talib.HT_TRENDLINE(data.close)
    data['mama'], data['fama'] = talib.MAMA(data.ht, fastlimit=params[0], slowlimit=params[0]/10)
    data['slope'] = talib.LINEARREG_SLOPE(data.close, timeperiod=params[1])
    data['tsf'] = talib.TSF(data.ht, params[2])
    data.dropna(axis=0, inplace=True)

    # signals
    data['mama_signals'] = np.where(data.mama > data.fama, 1, -1) 
    data['slope_signals'] = data.slope.apply(np.sign)
    data['tsf_signals'] = np.where(data.ht > data.tsf, 1, -1)
    signal_cols = ['mama_signals', 'slope_signals', 'tsf_signals']
    data['agg_signals'] = data[signal_cols].mode(axis=1)
    
    # train/test split
    train, test = (data.loc[:pd.Timestamp(split_date)-dt.timedelta(days=1)] , 
                   data.loc[pd.Timestamp(split_date):])
    
    # backtest train
    train_date_range = train.index[-1]-train.index[0]
    train_backtest = IterativeBacktester(data=train, signals=train.agg_signals, freq=freq)
    train_backtest.backtest(progress_bar=False)
    
    train_ret = train_backtest.return_df.loc['TotalReturn', 'Portfolio']
    train_signal_counts = train_backtest.signals.value_counts()
    train_signal_changes = train_backtest.signals.diff(1).dropna().apply(np.abs).value_counts()
    
    train_total_days = train_date_range.total_seconds() / (60*60*24)
    train_pos_short = train_signal_counts[-1]
    train_pos_long = train_signal_counts[1]
    train_pos_changes = (train_signal_changes.index * train_signal_changes).sum()
    
    # backtest test
    test_date_range = test.index[-1]-test.index[0]
    test_backtest = IterativeBacktester(data=test, signals=test.agg_signals, freq=freq)
    test_backtest.backtest(progress_bar=False)
    
    test_ret = test_backtest.return_df.loc['TotalReturn', 'Portfolio']
    test_signal_counts = test_backtest.signals.value_counts()
    test_signal_changes = test_backtest.signals.diff(1).dropna().apply(np.abs).value_counts()
    
    test_total_days = test_date_range.total_seconds() / (60*60*24)
    test_pos_short = test_signal_counts[-1]
    test_pos_long = test_signal_counts[1]
    test_pos_changes = (test_signal_changes.index * test_signal_changes).sum()
    
    # (train , test)
    # returns, #days, #short, #long, #posChanges
#     performances[params] = (train_ret, train_total_days, train_pos_short, train_pos_long, train_pos_changes,
#                             test_ret, test_total_days, test_pos_short, test_pos_long, test_pos_changes)
    return (params, 
            train_ret, train_total_days, train_pos_short, train_pos_long, train_pos_changes,
            test_ret, test_total_days, test_pos_short, test_pos_long, test_pos_changes)

In [53]:

params_list = []

for mama_period in np.arange(0.1, 1, 0.3):
    for slope_period in range(20, 101, 40):
        for tsf_period in range(20, 101, 40):
            params_list.append((mama_period, slope_period, tsf_period))
print(f'Set of Parameters: {len(params_list)}')

Set of Parameters: 27


In [14]:
split_date = '2022-07-17'
num_cpus = 4
results = p_umap(get_performance, 
                 repeat(data_sql), repeat(split_date), params_list, repeat(freq),
                 **{"num_cpus": num_cpus})

  0%|          | 0/27 [00:00<?, ?it/s]

In [41]:
res_df = pd.DataFrame(results)
res_df.columns = ['params', 
                  'train_returns', 'train_n_days', 'train_n_short', 'train_n_long', 'train_n_changes',
                  'test_returns', 'test_n_days', 'test_n_short', 'test_n_long', 'test_n_changes']
res_df.set_index('params', inplace=True)
res_df.sort_values(by='train_returns', ascending=False, inplace=True)

In [52]:
file_name = 'performances_train_test.xlsx'
if os.path.exists(file_name):
    mode = 'a'
else:
    mode = 'w'

engine = "openpyxl"
with pd.ExcelWriter(file_name, engine=engine, mode=mode) as writer:  
    workBook = writer.book
    try:
        workBook.remove(workBook[ticker.replace('/','_')])
    except:
        print("worksheet doesn't exist")
    finally:
        res_df.to_excel(writer, sheet_name=ticker.replace('/','_'), engine=engine)
    writer.save()